In [1]:
# Install required pacakges.
import torch as to
import numpy as np
import pandas as pd
import tqdm as tqdm

In [2]:
# Test out Sparse matrices.
i = [[0, 1, 1],
       [2, 0, 2]]
v = [3, 4, 5]
s = to.sparse_coo_tensor(i, v, (2, 3))
s

tensor(indices=tensor([[0, 1, 1],
                       [2, 0, 2]]),
       values=tensor([3, 4, 5]),
       size=(2, 3), nnz=3, layout=torch.sparse_coo)

In [3]:
s.to_dense()

tensor([[0, 0, 3],
        [4, 0, 5]])

In [4]:
s2 = s.coalesce()
s2.indices()

tensor([[0, 1, 1],
        [2, 0, 2]])

## ARP Build

In [5]:
# Building the sparse transition matrix.
# Start with action == 1
M = 3650
S = to.arange(1, M + 1)
A = to.arange(1, M + 1 + 1)
data = to.as_tensor(pd.read_csv('data/ARP_3650_data.csv').values)
print("The shape of this loaded data is...", data.shape)
c, v, o, p = to.hsplit(data, 4)

The shape of this loaded data is... torch.Size([3650, 4])


In [6]:
card_s = len(S)
card_a = len(A)

In [7]:
# the specified elements are two a row except for the last two
# columns: S = M, J = M ==> 1
#          S = M -1, J = M ==> 1
# Therefore, indices will be of size (M, 2)
indices = to.zeros(2, 2*3650)
indices.shape

torch.Size([2, 7300])

In [8]:
print('Number of CPUs available:', to.get_num_threads())
print('Number of CPUs to be used:', to.get_num_threads() - 2)

Number of CPUs available: 8
Number of CPUs to be used: 6


In [9]:
for s in S:
    if s < M - 1:
        indices[0, s - 1] = s - 1
        indices[1, s - 1] = s
    elif s == M - 1:
        indices[0, s - 1] = M - 1 - 1
        indices[1, s - 1] = M - 1
    elif s == M:
        indices[0, s - 1] = M - 1
        indices[1, s - 1] = M - 1

indices

tensor([[0., 1., 2.,  ..., 0., 0., 0.],
        [1., 2., 3.,  ..., 0., 0., 0.]])

In [10]:
for s in S:
    if s < M:
        indices[0, s + 3650 - 1] = s - 1
        indices[1, s + 3650 - 1] = M - 1
    else:
        indices[0, M + 3650 - 1] = s - 1
        indices[1, M + 3650 - 1] = M - 1

indices

tensor([[0.0000e+00, 1.0000e+00, 2.0000e+00,  ..., 3.6470e+03, 3.6480e+03,
         3.6490e+03],
        [1.0000e+00, 2.0000e+00, 3.0000e+00,  ..., 3.6490e+03, 3.6490e+03,
         3.6490e+03]])

In [11]:
indices = indices.type(to.IntTensor)
indices

tensor([[   0,    1,    2,  ..., 3647, 3648, 3649],
        [   1,    2,    3,  ..., 3649, 3649, 3649]], dtype=torch.int32)

In [12]:
indices[:, 3645: 3655]

tensor([[3645, 3646, 3647, 3648, 3649,    0,    1,    2,    3,    4],
        [3646, 3647, 3648, 3649, 3649, 3649, 3649, 3649, 3649, 3649]],
       dtype=torch.int32)

In [13]:
indices[:, 7295:]

tensor([[3645, 3646, 3647, 3648, 3649],
        [3649, 3649, 3649, 3649, 3649]], dtype=torch.int32)

In [14]:
values1 = [p[s-1].item() for s in S]
values1[-1] = 1
values1[-2] = 1
values2 = [1 - p[s-1].item() for s in S]
values2[-1] = 0
values2[-2] = 0

In [15]:
values = np.append(np.array(values1), np.array(values2))
values = values.reshape(1, 7300)

In [16]:
values[:, 3645 : 3655]

array([[4.3836e-03, 3.2877e-03, 2.1918e-03, 1.0000e+00, 1.0000e+00,
        1.0000e-05, 2.0000e-05, 3.0000e-05, 4.0000e-05, 5.0000e-05]])

In [17]:
P_action1 = to.sparse_coo_tensor(indices, values.reshape(7300, 1))
P_action1

tensor(indices=tensor([[   0,    1,    2,  ..., 3647, 3648, 3649],
                       [   1,    2,    3,  ..., 3649, 3649, 3649]]),
       values=tensor([[1.0000],
                      [1.0000],
                      [1.0000],
                      ...,
                      [0.9978],
                      [0.0000],
                      [0.0000]]),
       size=(3650, 3650, 1), nnz=7300, dtype=torch.float64,
       layout=torch.sparse_coo)

In [18]:
P_action1.to_dense().reshape(3650, 3650)

tensor([[0.0000e+00, 9.9999e-01, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e-05],
        [0.0000e+00, 0.0000e+00, 9.9998e-01,  ..., 0.0000e+00, 0.0000e+00,
         2.0000e-05],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         3.0000e-05],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 2.1918e-03,
         9.9781e-01],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00]], dtype=torch.float64)

In [19]:
i = 0
for row in P_action1.to_dense().reshape(3650, 3650):
    if sum(row) != 1:
        print('row {} does not sum to 1.'.format(i))
        
    i += 1

In [20]:
# BUILD ACTION 121 : purchase oldest car possible.
indices = to.zeros(2, 3650)
for s in S:
    indices[0, s - 1] = s - 1
    indices[1, s - 1] = M - 1

values = to.ones(3650, 1)
P_actionlast = to.sparse_coo_tensor(indices, values)
P_actionlast

tensor(indices=tensor([[   0,    1,    2,  ..., 3647, 3648, 3649],
                       [3649, 3649, 3649,  ..., 3649, 3649, 3649]]),
       values=tensor([[1.],
                      [1.],
                      [1.],
                      ...,
                      [1.],
                      [1.],
                      [1.]]),
       size=(3650, 3650, 1), nnz=3650, layout=torch.sparse_coo)

In [22]:
i = 0
for row in P_actionlast.to_dense().reshape(3650, 3650):
    if sum(row) != 1:
        print('row {} does not sum to 1.'.format(i))
        
    i += 1

In [23]:
# Build actions 2 through 3649.
# 2 non-zero elements each row.
indices = to.zeros(3648, 2, 3650*2)
indices.shape

torch.Size([3648, 2, 7300])

In [27]:
for a in tqdm.tqdm(range(2, 3649), desc='building first index set'):
    for s in S:
        indices[a-1, 0, s - 1] = s - 1
        indices[a-1, 1, s - 1] = a - 2 + 1

building first index set: 100%|██████████| 3647/3647 [03:48<00:00, 15.95it/s]


In [22]:
indices = to.zeros(2, 3650*2)
a = 2
for s in S:
    indices[0, s-1] = s - 1
    indices[1, s-1] = a - 2 + 1

for s in S:
    indices[0, s+3650-1] = s - 1
    indices[1, s+3650-1] = M - 1

values1 = to.from_numpy(np.repeat(p[a-2].item(), 3650))
values2 = to.from_numpy(np.repeat(1-p[a-2].item(), 3650))

In [25]:
values = np.append(values1.numpy(), values2.numpy())
values.shape

(7300,)

In [26]:
P_actionN = to.sparse_coo_tensor(indices, values.reshape(7300, 1))
P_actionN

tensor(indices=tensor([[   0,    1,    2,  ..., 3647, 3648, 3649],
                       [   1,    1,    1,  ..., 3649, 3649, 3649]]),
       values=tensor([[9.9999e-01],
                      [9.9999e-01],
                      [9.9999e-01],
                      ...,
                      [1.0000e-05],
                      [1.0000e-05],
                      [1.0000e-05]]),
       size=(3650, 3650, 1), nnz=7300, dtype=torch.float64,
       layout=torch.sparse_coo)

In [27]:
# First calculate how many non-zero elements are present in the transition probability matrix.
# Actions 1 through M --> 2 non-zero element per row
# Action M + 1 --> 1 non-zero element present
nonzero = 0
for a in A:
    if a < M + 1:
        nonzero += 2
    else:
        nonzero += 1

nonzero

7301

In [28]:
# Pre-allocate the memory for indices and values vectors.
indices = to.zeros(card_a, 2, nonzero*3650)